In [ ]:
!pip install pyts
!pip install --upgrade pyts

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import sys
from scipy import stats
from scipy.fft import fft, dct
import pywt
from sklearn.preprocessing import LabelEncoder
np.set_printoptions(threshold=sys.maxsize)
from matplotlib.backends.backend_pdf import PdfPages
import os
from pyts.approximation import PiecewiseAggregateApproximation as PAA
from scipy.fftpack import idct

def generate_random_vectors(n, r):
    random_vectors = np.random.normal(0, 1, size=(r, n))
    return random_vectors

def MINE(data,R,R_inv):

  prod = np.matmul(data,R)
  bitmap = np.where(prod < 0, 0, 1)
  recon = np.matmul(bitmap,R_inv)
  return recon


def plot_confidence_interval(scores,method,stride,compression,type):
    # compute mean silhouette score and confidence interval
    mean_score = np.mean(scores)
    ci = stats.t.interval(0.95, len(scores)-1, loc=mean_score, scale=stats.sem(scores))

    # 2D array with the lower and upper bounds of the confidence interval
    yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

    # create the figure and axis objects
    fig, ax = plt.subplots()

    # plot the data with error bars
    ax.plot(scores, marker='o')
    ax.errorbar(x=range(len(scores)), y=scores, yerr=yerr, fmt='none', ecolor='r')

    # set the axis labels and title
    ax.set_xlabel('Subset of features')

    if type == 'SC' :
     ax.set_ylabel('R2 score')
     if method == 'RAW':
      ax.set_title(f'R2-scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'R2-scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')


    if type == 'RMSE' :
     ax.set_ylabel('RMSE score')
     if method == 'RAW':
      ax.set_title(f'RMSE-scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'RMSE-scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')
    # return the figure object
    return fig

def plot_score_confidence_interval(data,window_size,compression):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4), sharey=True)  # Adjust the figure size as needed

    colors = ['b', 'g', 'r', 'c', 'm']  # Colors for each method

    #y_axis_limits = (-5, 5)  # Outlier limits

    for i, (scores, method) in enumerate(data):
        ax = axs[i]
        n = len(scores)
        x = np.arange(n) + 1
        y = np.array(scores)

        # Identify and count outliers based on the limits
        outliers = np.where((y < -5) | (y > 5))[0]
        num_outliers = len(outliers)

        # Exclude outliers from the data
        filtered_scores = np.delete(y, outliers)
        filtered_x = np.delete(x, outliers)

        mean = np.mean(filtered_scores)
        # Calculate the confidence interval
        ci = stats.t.interval(0.95, len(filtered_scores), loc=mean, scale=stats.sem(filtered_scores))
        # Plot the non-outlier data points and mean
        ax.plot(filtered_x, filtered_scores, 'o', markersize=4, color=colors[i])
        ax.plot(x, [mean] * n, '--', color=colors[i])

        fig.suptitle(f'R2-scores with 95% confidence interval // W= {window_size}, C= {compression} ')

        ax.set_xlabel('Window')
        ax.set_ylabel('Score')
        ax.set_title(f'{method} : [{ci[0]:.5f}, {ci[1]:.5f}] ')
        ax.grid(True)

        # Set the y-axis limits to the predefined range
        #ax.set_ylim(y_axis_limits)

        # Add the number of outliers as text in the top right corner
        ax.text(0.85, 0.9, f'Outliers: {num_outliers}', transform=ax.transAxes, fontsize=10, color='red', ha='right')

    return fig


def plot_RMSE_confidence_interval(data, window_size, compression, outlier_threshold=3):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4), sharey=True)  # Adjust the figure size as needed

    colors = ['b', 'g', 'r', 'c', 'm']  # Colors for each method

   # y_axis_limits = (-5, 5) # Outlier limits

    for i, (scores, method) in enumerate(data):
        ax = axs[i]
        n = len(scores)
        x = np.arange(n) + 1
        y = np.array(scores)
        # Identify and count outliers based on the limits
        outliers = np.where((y < -5) | (y > 5))[0]
        num_outliers = len(outliers)

        # Exclude outliers from the data
        filtered_scores = np.delete(y, outliers)
        filtered_x = np.delete(x, outliers)

        mean = np.mean(filtered_scores)
        # Calculate the confidence interval
        ci = stats.t.interval(0.95, len(filtered_scores), loc=mean, scale=stats.sem(filtered_scores))

        # Plot the non-outlier data points and mean
        ax.plot(filtered_x, filtered_scores, 'o', markersize=4, color=colors[i])
        ax.plot(x, [mean] * n, '--', color=colors[i])

        fig.suptitle(f'RMSE-scores with 95% confidence interval // W= {window_size}, C= {compression} ')

        ax.set_xlabel('Window')
        ax.set_ylabel('Score')
        ax.set_title(f'{method} : [{0}, {ci[1]:.5f}]')
        ax.grid(True)

        # Set the y-axis limits to the predefined range
       # ax.set_ylim(y_axis_limits)

        # Add the number of outliers as text in the top right corner
        ax.text(0.85, 0.9, f'Outliers: {num_outliers}', transform=ax.transAxes, fontsize=10, color='red', ha='right')

    return fig

# Load data
df = pd.read_csv('/content/drive/MyDrive/pump_sensor2.txt', delimiter=',')

# BROKEN,NORMAL,RECOVERING -> 0,0.5,1
le = LabelEncoder()
df.iloc[:, -1] = le.fit_transform(df.iloc[:, -1])
max_label = df.iloc[:, -1].max()
df.iloc[:, -1] = df.iloc[:, -1] / max_label * 1.0
# Discard the first row + first two columns
df = df.iloc[:, 2:]
data = df.to_numpy(dtype='float')
# Split  features (X) and target (y)
X = data[:, :-1]
#print(X)
y = data[:, -1]
#print(y[0])
# Fill Nan values based on the mean of each sensor
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)
# Select 1000 0.5 kai 1 labels
label_0_indices = np.where(y == 0)[0]
label_05_indices = np.where(y == 0.5)[0]
label_1_indices = np.where(y == 1)[0]
label_05_indices = np.random.choice(label_05_indices, size=1000, replace=False)
label_1_indices = np.random.choice(label_1_indices, size=1000, replace=False)
# Concatenate + shuffle
indices = np.concatenate((label_0_indices, label_05_indices, label_1_indices))
np.random.seed(123)   #Gia idio dataset shuffle se ola ta algorithms
np.random.shuffle(indices)
X= X[indices]
y= y[indices]

#print
# unique_labels, label_counts = np.unique(y, return_counts=True)
# for i in range(len(unique_labels)):
#    print(f"Label {unique_labels[i]}: {label_counts[i]}")
# X = imputer.fit_transform(X)

# window size, etc.
window_sizes = [16, 32, 64, 128]
compressions = [4, 8, 16]
methods = ['DFT', 'DCT', 'DWT', 'PAA','MINE']

#file for storing
folder_name = f"Linear Regression PUMP Measurements"
if not os.path.exists(folder_name):
   os.makedirs(folder_name)

raw_folder_name = f"RAW DATA"
if not os.path.exists(os.path.join(folder_name,raw_folder_name)):
   os.makedirs(os.path.join(folder_name,raw_folder_name))
pdf_file = os.path.join(folder_name,raw_folder_name, 'plots.pdf')

with PdfPages(pdf_file, 'a') as pdf:
  for window_size in window_sizes:

     # lists to store metrics for each window
     rmse_list = []
     #mae_list = []
     r2_list = []
     num =int( window_size+window_size/4)

     # Iterate over the windows
     # Iterate over the windows
     for i in range(0, len(X) - window_size * 2 + 1, window_size):
        # Split the data into training and testing sets
       X_train = X[i:i + window_size]
       X_test = X[i + window_size:i + num]
       y_train = y[i:i + window_size]
       y_test = y[i + window_size:i + num]

        # Data scaling
       sc = MinMaxScaler()
       X_train_scaled = sc.fit_transform(X_train)
       X_test_scaled = sc.transform(X_test)

       # Model application
       reg_line = LinearRegression()
       reg_line.fit(X_train_scaled, y_train)
        # Generate predictions
       y_pred_scaled = reg_line.predict(X_test_scaled)

       # performance (mean squared error and R-squared score)
       mse = mean_squared_error(y_test, y_pred_scaled)
       rmse = np.sqrt(mse)
       r2 = r2_score(y_test, y_pred_scaled)

       rmse_list.append(rmse)
       r2_list.append(r2)

     #store plots
     methd = 'RAW'
     comprsn = 0
     fig = plot_confidence_interval(r2_list,methd,window_size ,comprsn,'SC') #plot function call
     pdf.savefig(fig)
     plt.close(fig)

     fig2 = plot_confidence_interval(rmse_list,methd,window_size ,comprsn,'RMSE') #plot function call
     pdf.savefig(fig2)
     plt.close(fig2)

     # create a file name based on the method, subset index, stride, and compression
     avg_r2 = np.mean(r2_list)
     file_name = f"Window size({window_size}) - R2-scores.txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average R2 Score: {avg_r2}\n")
              content = str(r2_list)
              f.write(content)
              f.close()

     avg_rmse = np.mean(rmse_list)
     file_name = f"Window size({window_size}) - RMSE scores).txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average RMSE Score: {avg_rmse}\n")
              content = str(rmse_list)
              f.write(content)
              f.close()

for window_size in window_sizes:

    #file for storing
    stride_folder_name = f"stride({window_size})"
    if not os.path.exists(os.path.join(folder_name,stride_folder_name)):
        os.makedirs(os.path.join(folder_name,stride_folder_name))

    for compression in compressions:

        #vector and inverse vector arrays for our algorithm
        n = 64*int(window_size/compression)  # Dimension of each random vector
        r = 51  # Number of random vectors to generate/rows
        R = generate_random_vectors(n, r)
        R_inv= np.linalg.pinv(R)

        #files for storing measurements
        compression_folder_name = f"compression({compression})"
        if not os.path.exists(os.path.join(folder_name,stride_folder_name, compression_folder_name)):
            os.makedirs(os.path.join(folder_name,stride_folder_name, compression_folder_name))

        #files for storing plots
        plots_dir = os.path.join(folder_name,stride_folder_name, compression_folder_name, 'plots')
        os.makedirs(plots_dir, exist_ok=True)
        pdf_file = os.path.join(plots_dir, 'plots.pdf')

        with PdfPages(pdf_file, 'a') as pdf:
            results1 = []
            results2 = []
            for method in methods:

                slices = window_size // compression

                # lists to store metrics for each window
                rmse_list = []
                #mae_list = []
                r2_list = []
                num =int( window_size+window_size/4)

                # Iterate over the windows
                for i in range(0, len(X) - window_size * 2 + 1, window_size):
                   # Split the data into training and testing sets
                   X_train = X[i:i + window_size]
                   X_test = X[i + window_size:i + num]
                   y_train = y[i:i + window_size]
                   y_test = y[i + window_size:i + num]

                   # Data scaling
                   sc = MinMaxScaler()
                   X_train_scaled = sc.fit_transform(X_train)
                   X_test_scaled = sc.transform(X_test)

                   #print(X_train_scaled[0], X_test_scaled[0], y_train_scaled[0], y_test_scaled[0])

                   if method == 'DFT':

                       #array to store the restored data
                       compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                       for i in range(window_size):
                          # Compute abs DFT of each row
                          dft_X_train_scaled  =  np.fft.fft(X_train_scaled[i])
                          # Sort the DFT coefficients along each row by their magnitude
                          sorted_indices = np.argsort(-np.abs(dft_X_train_scaled))[:slices]
                          sorted_dft_X_train_scaled = dft_X_train_scaled[sorted_indices]
                          # Keep  top  coeff
                          top__dft_X_train_scaled = sorted_dft_X_train_scaled[:slices]
                          #reconstruct the compressed dataset
                          compressed_X_train_scaled[i] = np.fft.ifft(top__dft_X_train_scaled,51).real

                       #array to store the restored data
                       compressed_X_test_scaled = np.zeros_like(X_test_scaled)

                       for i in range(int(window_size/4)):
                          # Compute abs DFT of each row
                          dft_X_test_scaled =  np.fft.fft(X_test_scaled[i])
                          # Sort the DFT coefficients along each row by their magnitude
                          sorted_indices = np.argsort(np.abs(dft_X_test_scaled))[::-1][:slices]
                          sorted_dft_X_test_scaled = dft_X_test_scaled[sorted_indices]
                          # Keep  top  coeff
                          top__dft_X_test_scaled = sorted_dft_X_test_scaled[:slices]
                          #reconstruct the compressed dataset
                          compressed_X_test_scaled[i] = np.fft.ifft(top__dft_X_test_scaled,51).real

                   elif method == 'DCT':

                       #array to store the restored data
                       compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                       for i in range(window_size):
                          # Compute abs DCT of each row
                          dct_X_train_scaled  =  dct(X_train_scaled[i])
                          # Sort the DCT coefficients along each row by their magnitude
                          sorted_indices = np.argsort(-np.abs(dct_X_train_scaled))[:slices]
                          sorted_dct_X_train_scaled = dct_X_train_scaled[sorted_indices]
                          # Keep  top  coeff
                          top__dct_X_train_scaled = sorted_dct_X_train_scaled[:slices]
                          #reconstruct the compressed dataset
                          compressed_X_train_scaled[i] = idct(top__dct_X_train_scaled ,type = 2, n = 51).real

                       #array to store the restored data
                       compressed_X_test_scaled = np.zeros_like(X_test_scaled)

                       for i in range(int(window_size/4)):
                          # Compute abs DCT of each row
                          dct_X_test_scaled =  dct(X_test_scaled[i])
                          # Sort the DCT coefficients along each row by their magnitude
                          sorted_indices = np.argsort(-np.abs(dct_X_test_scaled))[:slices]
                          sorted_dct_X_test_scaled = dct_X_test_scaled[sorted_indices]
                          # Keep  top  coeff
                          top__dct_X_test_scaled = sorted_dct_X_test_scaled[:slices]
                          #reconstruct the compressed dataset
                          compressed_X_test_scaled[i] = idct(top__dct_X_test_scaled ,type = 2 , n = 51).real


                   elif method == 'DWT':

                      #array to store the restored data
                      compressed_X_train_scaled = np.zeros_like(X_train_scaled)
                      compressed_X_train_scaled = np.pad(compressed_X_train_scaled, ((0, 0), (0, 1)), 'constant', constant_values=0)
                      for i in range(window_size):
                          # Apply DWT to each row using 'db1' wavelet
                          cA, cD = pywt.dwt(X_train_scaled[i], 'db1')
                          # Sort
                          sorted_cD_subset_data = np.zeros_like(cD) #initialize storage array
                          sorted_indices = np.argsort(-np.abs(cD))[:slices]
                          sorted_cD_subset_data[sorted_indices] = cD[sorted_indices]
                          # Perform inverse DWT to restore the row
                          compressed_X_train_scaled[i] = pywt.idwt(cA, sorted_cD_subset_data, 'db1')

                      #array to store the restored data
                      compressed_X_test_scaled = np.zeros_like(X_test_scaled)
                      compressed_X_test_scaled = np.pad(compressed_X_test_scaled, ((0, 0), (0, 1)), 'constant', constant_values=0)
                      for i in range(int(window_size/4)):
                          # Apply DWT to each row using 'db1' wavelet
                          cA, cD = pywt.dwt(X_test_scaled[i], 'db1')
                          # Sort
                          sorted_cD_subset_data = np.zeros_like(cD) #initialize storage array
                          sorted_indices = np.argsort(-np.abs(cD))[:slices]
                          sorted_cD_subset_data[sorted_indices] = cD[sorted_indices]
                          # Perform inverse DWT to restore the row
                          compressed_X_test_scaled[i] = pywt.idwt(cA, sorted_cD_subset_data, 'db1')


                   elif method == 'PAA':
                      # Apply PAA along the rows of the array
                      paa = PAA(window_size = compression)
                      compressed_X_train_scaled = paa.fit_transform(X_train_scaled)
                      compressed_X_test_scaled = paa.fit_transform(X_test_scaled)

                   elif method == 'MINE':

                      #array to store the restored data
                      compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                      for i in range(window_size):
                        row = X_train_scaled[i]
                        # Apply my algorithm along the rows of the array
                        compressed_X_train_scaled[i] = MINE(row,R,R_inv)

                      #array to store the restored data
                      compressed_X_test_scaled = np.zeros_like(X_test_scaled)
                      for i in range(int(window_size/4)):
                        row = X_test_scaled[i]
                        # Apply my algorithm along the rows of the array
                        compressed_X_test_scaled[i] = MINE(row,R,R_inv)

                   # Model application
                   regressor = LinearRegression()
                   regressor.fit(compressed_X_train_scaled, y_train)

                   # Generate predictions
                   y_pred_scaled = regressor.predict(compressed_X_test_scaled)

                   # Transform scaled predictions back to og scale
                    #y_pred = sc.inverse_transform(y_pred_scaled)

                   # performance (mean squared error, mean absolute error, and R-squared score)
                   mse = mean_squared_error(y_test, y_pred_scaled)
                   rmse = np.sqrt(mse)
                    #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
                   r2 = r2_score(y_test, y_pred_scaled)

                   # Add to list of measurements
                   rmse_list.append(rmse)
                   #mae_list.append(mae)
                   r2_list.append(r2)

                results1.append((r2_list, method))
                results2.append((rmse_list, method))

                # create a file name based on the method, subset index, stride, and compression
                avg_r2 = np.mean(r2_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - R2 SCORES).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average R2-Score: {avg_r2}\n")
                              content = str(r2_list)
                              f.write(content)
                              f.close()

                avg_RMSE = np.mean(rmse_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - RMSEs).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average RMS-Score: {avg_RMSE}\n")
                              content = str(rmse_list)
                              f.write(content)
                              f.close()
            #store plots
            fig = plot_score_confidence_interval(results1,window_size,compression) #plot function call
            pdf.savefig(fig)
            plt.close(fig)
            fig = plot_RMSE_confidence_interval(results2,window_size,compression) #plot function call
            pdf.savefig(fig)
            plt.close(fig)










<ipython-input-3-c8585b444d62>:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, -1] = le.fit_transform(df.iloc[:, -1])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import sys
from scipy import stats
from scipy.fft import fft, dct
import pywt
from sklearn.preprocessing import LabelEncoder
np.set_printoptions(threshold=sys.maxsize)

def paa(data, n_pieces):
    """
    Piecewise Aggregate Approximation (PAA) on data.
    """
    n = data.shape[0]
    piece_length = int(np.ceil(n/n_pieces))
    padded_data = np.pad(data, ((0, piece_length*n_pieces-n), (0,0)), mode='constant', constant_values=0)
    pieces = padded_data.reshape(n_pieces, piece_length, -1)
    return np.mean(pieces, axis=1)



# Load data
df = pd.read_csv('/content/drive/MyDrive/pump_sensor2.txt', delimiter=',')

# BROKEN,NORMAL,RECOVERING -> 0,0.5,1
le = LabelEncoder()
df.iloc[:, -1] = le.fit_transform(df.iloc[:, -1])
max_label = df.iloc[:, -1].max()
df.iloc[:, -1] = df.iloc[:, -1] / max_label * 1.0
# Discard the first row + first two columns
df = df.iloc[:, 2:]
data = df.to_numpy(dtype='float')
# Split  features (X) and target (y)
X = data[:, :-1]
#print(X)
y = data[:, -1]
#print(y[0])
# Fill Nan values based on the mean of each sensor
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)
# Select 1000 0.5 kai 1 labels
label_0_indices = np.where(y == 0)[0]
label_05_indices = np.where(y == 0.5)[0]
label_1_indices = np.where(y == 1)[0]
label_05_indices = np.random.choice(label_05_indices, size=1000, replace=False)
label_1_indices = np.random.choice(label_1_indices, size=1000, replace=False)
# Concatenate + shuffle
indices = np.concatenate((label_0_indices, label_05_indices, label_1_indices))
np.random.seed(123)   #Gia idio dataset shuffle se ola ta algorithms
np.random.shuffle(indices)
X= X[indices]
y= y[indices]
#print
# unique_labels, label_counts = np.unique(y, return_counts=True)
# for i in range(len(unique_labels)):
#    print(f"Label {unique_labels[i]}: {label_counts[i]}")
# X = imputer.fit_transform(X)

# window size, etc.
window_size = 128 #@param {type:"number"}
method = 4              #@param {type:"number"} 1=DFT , 2=DCT, 3=discrete wavelet transform , 4=PAA
num = 0
num =int( window_size+window_size/4)

# lists to store metrics for each window
mse_list = []
mae_list = []
r2_list = []
subset_number = 1

# Iterate over the windows
for i in range(0, len(X)-window_size*2+1, window_size):

 # Split the data into training and testing sets
 X_train = X[i:i+window_size]
 X_test = X[i+window_size:i+num]
 y_train = y[i:i+window_size]
 y_test = y[i+window_size:i+num]
 print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

 # Data scaling
 sc = MinMaxScaler()
 X_train_scaled = sc.fit_transform(X_train)
 X_test_scaled = sc.transform(X_test)
 y_train_scaled = sc.fit_transform (y_train.reshape(y_train.shape[0],1))
 y_test_scaled = sc.transform (y_test.reshape(y_test.shape[0],1))
 #print(X_train_scaled[0], X_test_scaled[0], y_train_scaled[0], y_test_scaled[0])

 if method == 1:
        X_train_scaled = np.abs(fft(X_train_scaled, axis=0))
        X_test_scaled = np.abs(fft(X_test_scaled, axis=0))
        y_train_scaled = np.abs(fft(y_train_scaled, axis=0))
        y_test_scaled = np.abs(fft(y_test_scaled, axis=0))
        print_string = 'DFT'
 elif method == 2:
        X_train_scaled = dct(X_train_scaled, axis=0)
        X_test_scaled = dct(X_test_scaled, axis=0)
        y_train_scaled = dct(y_train_scaled, axis=0)
        y_test_scaled = dct( y_test_scaled, axis=0)
        print_string = 'DCT'
 elif method == 3:
        cA, cD = pywt.dwt(X_train_scaled, 'db1', axis=0)
        X_train_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(X_test_scaled, 'db1', axis=0)
        X_test_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(y_train_scaled, 'db1', axis=0)
        y_train_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(y_test_scaled, 'db1', axis=0)
        y_test_scaled = np.concatenate((cA, cD), axis=0)
        print_string = 'DWT'
 elif method == 4:
        X_train_scaled = paa(X_train_scaled, n_pieces=8)
        X_test_scaled = paa(X_test_scaled, n_pieces=8)
        y_train_scaled = paa(y_train_scaled, n_pieces=8)
        y_test_scaled = paa( y_test_scaled, n_pieces=8)
        print_string = 'PAA'
 else   :
      print_string = 'RAW'

 # Model application
 reg_line = LinearRegression()
 reg_line.fit(X_train_scaled, y_train_scaled)

 # Generate predictions
 y_pred_scaled = reg_line.predict(X_test_scaled)

 # Transform scaled predictions back to og scale
 #y_pred = sc.inverse_transform(y_pred_scaled)

 # performance (mean squared error, mean absolute error, and R-squared score)
 mse = mean_squared_error(y_test_scaled, y_pred_scaled)
 mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
 r2 = r2_score(y_test_scaled, y_pred_scaled)

 #print(f"Subset {subset_number} : | MSE= {mse:.5f}, | MAE= {mae:.5f}, R-squared= {r2:.5f}")
 subset_number= subset_number + 1
 # Add to list of measurements
 mse_list.append(mse)
 mae_list.append(mae)
 r2_list.append(r2)

# Saving the scores array in a text file
file_name = f"Linear Regression({window_size}) {print_string} measurements.txt"
file = open(file_name, "w+")
content = str(r2_list)
file.write(content)
file.close()

#plot

# mean silhouette score and confidence interval
mean_score = np.mean(r2_list)
ci = stats.t.interval(0.95, len(r2_list)-1, loc=mean_score, scale=stats.sem(r2_list))

# 2D array with the lower and upper bounds of the confidence interval
yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

# Plot the data with error bars
fig, ax = plt.subplots()
ax.plot(r2_list, marker='o')
ax.errorbar(x=range(len(r2_list)), y=r2_list, yerr=yerr, fmt='none', ecolor='r')
ax.set_xlabel('Subset of features')
ax.set_ylabel('R2 score')
ax.set_title('R2 scores with 95% confidence interval')
plt.show()

print(f"95% confidence interval: [{ci[0]:.5f}, {ci[1]:.5f}]")



In [ ]:
from scipy import stats
 #plot

# mean silhouette score and confidence interval
mean_score = np.mean(r2_list)
ci = stats.t.interval(0.95, len(r2_list)-1, loc=mean_score, scale=stats.sem(r2_list))

# 2D array with the lower and upper bounds of the confidence interval
yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

# Plot the data with error bars
fig, ax = plt.subplots()
ax.plot(r2_list, marker='o')
ax.errorbar(x=range(len(r2_list)), y=r2_list, yerr=yerr, fmt='none', ecolor='r')
ax.set_xlabel('Subset of features')
ax.set_ylabel('R2 score')
ax.set_title('R2 scores with 95% confidence interval')
plt.show()

print(f"95% confidence interval: [{ci[0]:.5f}, {ci[1]:.5f}]")

In [ ]:
#@title Whole file

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import sys

np.set_printoptions(threshold=sys.maxsize)

# Load data
df = pd.read_csv('/content/drive/MyDrive/pump_sensor2.txt', delimiter=',')

# BROKEN,NORMAL,RECOVERING -> 0,1,2
le = LabelEncoder()
df.iloc[:, -1] = le.fit_transform(df.iloc[:, -1])

# Discard the first row + first two columns
df = df.iloc[:, 2:]

data = df.to_numpy(dtype='float')

# Split  features (X) and target (y)
X = data[:, :-1]
#print(X)
y = data[:, -1]
#print(y[0])

# Fill Nan values based on the mean of each sensor
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Data scaling
sc = MinMaxScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

y_train_scaled = sc.fit_transform (y_train.reshape(y_train.shape[0],1))
y_test_scaled = sc.transform (y_test.reshape(y_test.shape[0],1))
#print(X_train_scaled[0], X_test_scaled[0], y_train_scaled[0], y_test_scaled[0])

# Model application
reg_line = LinearRegression()
reg_line.fit(X_train_scaled, y_train_scaled)

# Generate predictions
y_pred_scaled = reg_line.predict(X_test_scaled)

# Transform scaled predictions back to og scale
y_pred = sc.inverse_transform(y_pred_scaled)

# Performance (mean squared error, mean absolute error, and R-squared score)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE: ", mse)
print("MAE: ", mae)
print("R-squared: ", r2)

# Plot the predicted values against the actual values
plt.scatter(y_test, y_pred)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.title("Linear Regression Results")
plt.show()

